# Data Analysis Workbook

In [67]:
import os
import pymysql
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout, Activation
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
sklearn.__version__

'0.23.2'

In [3]:
conn = pymysql.connect(
    host='localhost',
    port=int(3306),
    user="root",
    passwd='vaughn',
    db="Texas_Education_Data",
    charset='utf8mb4')

In [4]:
def get_rolling_df(df, window=4):
    main_rolling_df = pd.DataFrame(columns=df.columns)
    for district in df.District_Id.unique():
        district_df = df[df.District_Id == district]
        
        rolling_df = district_df.rolling(window).mean()
        rolling_df['Year'] = district_df['Year'].rolling(window).max()

        main_rolling_df = pd.concat([main_rolling_df, rolling_df])

    main_rolling_df = main_rolling_df.dropna(how='all')
    return main_rolling_df

#### Approximate district affluence by dividing local revenue by number of students

In [5]:
query = "select e.District_Id, e.Year, e.Fall_Enrollment, r.Total_Local_Revenue_All_Funds as Local_Revenue, r.Total_State_Revenue_All_Funds as State_Revenue from Enrollment e, Revenue r where e.District_Id = r.District_Id and e.Year = r.Year"
affluence_df = pd.read_sql_query(query,conn)

affluence_df = affluence_df.set_index(['District_Id', 'Year']).dropna()

affluence_df['Local_Per_Student'] = affluence_df['Local_Revenue'] / affluence_df['Fall_Enrollment']
affluence_df['State_Per_Student'] = affluence_df['State_Revenue'] / affluence_df['Fall_Enrollment']

affluence_df = affluence_df.loc[affluence_df['Local_Revenue'] > 0].loc[affluence_df['Fall_Enrollment'] > 0]
affluence_df = affluence_df.loc[affluence_df['State_Revenue'] > 0].loc[affluence_df['Fall_Enrollment'] > 0]

# rolling_affluence_df = get_rolling_df(affluence_df.reset_index()).set_index(['District_Id','Year'])
# rolling_affluence_df
# affluence_df['Local_Per_Student'], affluence_bins = pd.qcut(affluence_df['Local_Per_Student'], 50, labels=False, retbins=True, duplicates='drop')

#### Get class teacher salary information

In [6]:
query = "select District_Id, Year, Num_Employees, Total_Employee_Pay from Teachers"
salary_df = pd.read_sql_query(query,conn)
salary_df = salary_df.groupby(['District_Id','Year']).sum()
# salary_df['Avg_Salary'] = salary_df['Total_Employee_Pay'] / salary_df['Num_Employees']
# rolling_salary_df = get_rolling_df(salary_df.reset_index()).set_index(['District_Id','Year'])
# rolling_salary_df

#### Get exponential weighted average of test scores

In [7]:
query = "select * from Test_Scores"
test_scores_df = pd.read_sql_query(query,conn).set_index(['District_Id', 'Year'])

In [8]:
# rolling_average_scores = pd.DataFrame(columns=test_scores_df.columns).set_index(['District_Id','Year'])
# for district in test_scores_df.District_Id.unique():
#     district_df = test_scores_df[test_scores_df.District_Id == district].set_index(['District_Id','Year'])
#     rolling_df = district_df.ewm(span=20, adjust=False).mean()
    
#     rolling_average_scores = pd.concat([rolling_average_scores, rolling_df])

#### Break % above critical ratio sat or act in quantiles

In [9]:
# sat_act_crit_rate = rolling_average_scores['Above_Crit_Rate_Sat_Act'].dropna().reset_index()
# sat_act_crit_rate = sat_act_crit_rate.loc[sat_act_crit_rate.Year > 2007].set_index(['District_Id','Year'])
# test_score_quartiles,bins = pd.qcut(sat_act_crit_rate['Above_Crit_Rate_Sat_Act'], 4, labels=False, retbins=True)

#### Get class size data

In [10]:
query = "select District_Id, Year, Num_Teachers, Num_Students from Classes"
class_size_df = pd.read_sql_query(query,conn)
class_size_df = class_size_df.groupby(['District_Id', 'Year']).sum()
class_size_df['Class_Size'] = class_size_df['Num_Students'] / class_size_df['Num_Teachers']

#### Get expenditures by function with 4 year simple moving average

In [11]:
query = "select * from Functions"
functions_df = pd.read_sql_query(query,conn).set_index(['District_Id', 'Year'])
# rolling_functions_df = get_rolling_df(functions_df).set_index(['District_Id','Year'])

#### Get enrollment

In [12]:
query = "select * from Enrollment"
enrollment_df = pd.read_sql_query(query,conn).set_index(['District_Id', 'Year'])
funct_per_student = pd.concat([functions_df, enrollment_df], axis=1)
funct_per_student = funct_per_student.div(funct_per_student['Fall_Enrollment'],axis=0).drop(['Fall_Enrollment'],axis=1)

#### Convert $ to % of all funds budget and add critical ratio to df

In [13]:
# rolling_functions_df = rolling_functions_df.filter(regex='.*_General_Funds$')
# func_all_funds_percent = pd.concat([rolling_functions_df, rolling_affluence_df, test_score_quartiles],axis=1).dropna().rename(columns={'Above_Crit_Rate_Sat_Act':'target'})
# func_all_funds_percent
# pd.concat([functions_df, class_size_df, test_scores_df, salary_df, affluence_df], axis=1)
dataset = pd.concat([funct_per_student, test_scores_df['Avg_Act'].to_frame()], axis=1).reset_index()
dataset = dataset[dataset.Year > 2004]

#remove districts without data between 2005-2019
for dist in dataset.District_Id.unique():
    if len(dataset[dataset.District_Id == dist].Year.unique()) != len(range(2005,2020)):
        dataset = dataset[dataset.District_Id != dist]

#remove district with less than 60% of act scores reported
for dist in dataset.District_Id.unique():
    dist_df = dataset[dataset.District_Id == dist]
    tot_rows = dist_df.shape[0]
    act_rows = dist_df[dist_df.Avg_Act.notna()].shape[0]
    if act_rows / tot_rows < .6:
        dataset = dataset[dataset.District_Id != dist]

#interpolate missing values
for dist in dataset.District_Id.unique():
    dist_df = dataset[dataset.District_Id == dist]
    dataset[dataset.District_Id == dist] = dist_df.interpolate(limit_direction='both')

dataset = dataset.filter(regex='(.*_General_Funds$)|(Avg_Act)|(District_Id)|(Year)')

# dataset[['Avg_Act','Total_Expenditure_By_Function_General_Funds']].plot.scatter(x='Total_Expenditure_By_Function_General_Funds',y='Avg_Act')
# dataset[['Avg_Act','Instruction_General_Funds']].plot.scatter(x='Instruction_General_Funds',y='Avg_Act')
# dataset[['Avg_Act','Security_Monitoring_General_Funds']].plot.scatter(x='Security_Monitoring_General_Funds',y='Avg_Act')
# dataset[['Avg_Act','Food_General_Funds']].plot.scatter(x='Food_General_Funds',y='Avg_Act')
# dataset[['Avg_Act','Community_Services_General_Funds']].plot.scatter(x='Community_Services_General_Funds',y='Avg_Act')
# dataset[['Avg_Act','Guidance_Counseling_Services_General_Funds']].plot.scatter(x='Guidance_Counseling_Services_General_Funds',y='Avg_Act')
# dataset[['Avg_Act','Curriculum_Staff_Develop_General_Funds']].plot.scatter(x='Curriculum_Staff_Develop_General_Funds',y='Avg_Act')


In [14]:
def split_windows(df, window_size=4):
    inputs = np.empty((0, window_size, len(dataset.columns)))
    labels = np.array([])
    t = 0
    for i in range(len(df.Year.unique()) - window_size + 1):
        start_year = 2005
        
        window = df.loc[df.Year >= start_year + i].loc[df.Year < start_year + i + window_size]
        ids = len(window.District_Id.unique())
        window = np.array(window)
        
        window_inputs = np.array(np.array_split(window, ids))
        window_labels = window_inputs[:,-1,-1].copy()
        window_inputs[:,-1,-1] = 1
        
        inputs = np.concatenate((inputs, window_inputs),axis=0)
        
        labels = np.append(labels,[window_labels])
    
    return inputs, labels

label_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_labels = label_scaler.fit_transform(np.array(dataset)[:,-1].reshape(-1,1))

feature_scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = feature_scaler.fit_transform(np.array(dataset)[:,2:-1])

scaled_dataset = pd.DataFrame(np.hstack((np.array(dataset)[:,:2], scaled_features, scaled_labels)), columns=dataset.columns)

window = 4
inputs, labels = split_windows(scaled_dataset, window)

def interleave_windows(data, window):
    empty = np.empty((data.shape))
    s = int(data.shape[0] / window)
    
    for i in range(window):
        a = inputs[i*s:(i+1)*s]
        empty[i::window] = a
    
    return empty

inputs = interleave_windows(inputs,window)

X = np.array([i.mean(axis=0) for i in inputs])
y, bins = pd.qcut(labels, 4, labels=False, retbins=True)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,shuffle=False)

clf = tree.DecisionTreeClassifier()
clf = RandomForestClassifier(n_estimators=100, bootstrap=True)
clf = clf.fit(X_train, y_train)
pred = clf.predict(X_test)

print(accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

0.4180253623188406
[[285 147  69  64]
 [127 195 145  88]
 [ 77 154 164 133]
 [ 50  87 144 279]]
              precision    recall  f1-score   support

           0       0.53      0.50      0.52       565
           1       0.33      0.35      0.34       555
           2       0.31      0.31      0.31       528
           3       0.49      0.50      0.50       560

    accuracy                           0.42      2208
   macro avg       0.42      0.42      0.42      2208
weighted avg       0.42      0.42      0.42      2208



In [47]:
splits = (np.array([.6,1]) * inputs.shape[0]).astype(int) 

train_inputs = inputs[:splits[0],:,:]
test_inputs = inputs[splits[0]:splits[1],:,:]

train_labels = labels[:splits[0]]
test_labels = labels[splits[0]:splits[1]]

train_inputs = train_inputs[:,:,2:]#.reshape(train_inputs.shape[0],8,1)
test_inputs = test_inputs[:,:,2:]#.reshape(test_inputs.shape[0],8,1)

train_dataset = tf.data.Dataset.from_tensor_slices((train_inputs, train_labels)).shuffle(buffer_size=1024).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((test_inputs, test_labels)).shuffle(buffer_size=1024).batch(32)

In [61]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units = 64, return_sequences = True, input_shape = (4, 18)))
model.add(Dropout(0.2))

model.add(tf.keras.layers.LSTM(units = 64, return_sequences = True))
model.add(Dropout(0.2))

# model.add(tf.keras.layers.LSTM(units = 64, return_sequences = True))
# model.add(Dropout(0.2))

model.add(tf.keras.layers.LSTM(units = 64, return_sequences = False))
model.add(Dropout(0.2))

model.add(Dense(units = 1))

model.compile(optimizer = tf.optimizers.Adam(), loss = tf.losses.MeanSquaredError(), metrics=[tf.metrics.MeanAbsoluteError()])
# model.fit(train_dataset, epochs = 5)
model.fit(np.random.rand(train_inputs.shape[0],train_inputs.shape[1],train_inputs.shape[2]),train_labels, epochs = 5, batch_size=32)


Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
207/207 [==============================] - 1s 5ms/step - loss: 0.0158 - mean_absolute_error: 0.0957
Epoch 2/5
207/207 [==============================] - 1s 4ms/step - loss: 0.0110 - mean_absolute_error: 0.0829
Epoch 3/5
207/207 [==============================] - 1s 4ms/step - loss: 0.0110 - mean_absolute_error: 0.0827
Epoch 4/5
207/207 [==============================] - 1s 4m

In [62]:
predictions = model.predict(test_inputs)
predictions = label_scaler.inverse_transform(predictions)
actual = label_scaler.inverse_transform(test_labels.reshape(1,-1))
np.sqrt(np.mean((predictions - actual)**2))
# list(np.hstack((actual.T, predictions)))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: Bad argument number for Name: 4, expecting 3
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2.1566528520153083

In [ ]:
plt.plot(actual, range(actual), label = "line 1")